In [2]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
from rnn_tools.model_saver import model_saver, model_saver_wandb
import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
import rnn_tools.training_testing_lstm as training_testing_lstm
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_gru_ensemble as training_testing_gru_ensemble
import rnn_tools.training_testing_gru_ensemble_pretrained as training_testing_gru_ensemble_pretrained

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [ ]:
torch.__version__

'2.1.1'

In [ ]:
['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']

['dist_last__1',
 'box_last__1',
 'speed_avg_1',
 'split_speed_v1_1',
 'split_speed_avg_1',
 'split_margin_avg_1',
 'margin_avg_1',
 'margin_time_avg_1',
 'RunHomeTime_1',
 'run_home_speed_1',
 'run_home_speed_v1_1',
 'first_out_avg_1',
 'pos_out_avg_1',
 'post_change_avg_1',
 'races_1',
 'wins_1',
 'wins_last_1',
 'weight_',
 'min_time_',
 'min_split_time_',
 'min_split_time_v1',
 'last_start_price',
 'last_start_prob']

In [ ]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [ ]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [ ]:
tracks = ['Sandown Park',
 'The Meadows',
 'Warragul',
 'Geelong',
 'Sale',
 'Ballarat',
 'Bendigo',
 'Cranbourne']

In [ ]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                             
date = datetime.datetime.strptime("2022-01-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ","NSW","QLD","NT","WA","SA","TAS"]
data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
raceDB = rnn_tools.raceDB.build_dataset(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python
[['inside', 'midfield', 'wide', 'weight', 'DogGrade', '_dist_last__1', '_box_last__1', '_speed_avg_1', '_split_speed_avg_1', '_split_margin_avg_1', '_margin_avg_1', '_margin_time_avg_1', '_RunHomeTime_1', '_run_home_speed_1', '_first_out_avg_1', '_pos_out_avg_1', '_post_change_avg_1', '_races_1', '_wins_1', '_wins_last_1', '_weight_', '_min_time_', '_min_split_time_', '_last_start_price', '_last_start_prob', 'dist_dist_last__10', 'dist_box_last__10', 'dist_speed_avg_10', 'dist_split_speed_avg_10', 'dist_split_margin_avg_10', 'dist_margin_avg_10', 'dist_margin_time_avg_10', 'dist_RunHomeTime_10', 'dist_run_home_speed_10', 'dist_first_out_avg_10', 'dist_pos_out_avg_10', 'dist_post_change_avg_10', 'dist_races_10', 'dist_wins_10', 'dist_wins_last_10', 'dist_weight_', 'dist_min_time_', 'dist_min_split_time_', 'dist_last_start_price', 'dist_last_start_prob', 'box_dist_last__10', 'box_box_last__10', 'box_speed_avg_10', 'box_split_speed_avg_10', '

100%|██████████| 25/25 [00:00<?, ?it/s]
c:\Users\Nick\Documents\GitHub\grvmodel\python\rnn_tools\rnn_classes.py:389: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r.prob = torch.tensor((1/(torch.tensor(r.start_prices)+0.0001))/((1/torch.tensor(r.start_prices)).sum()+0.0001),device=self.device)


number of races = 77666, number of unique dogs = 24858
0        (Mount Gambier, 400.0)
1                (Dapto, 520.0)
2           (Townsville, 380.0)
3             (Richmond, 401.0)
4          (Albion Park, 331.0)
                  ...          
77661         (Gunnedah, 340.0)
77662            (Nowra, 365.0)
77663     (Christchurch, 520.0)
77664            (Nowra, 365.0)
77665          (Ipswich, 288.0)
Length: 77666, dtype: object


In [ ]:
raceDB.stats_cols = eval(raceDB.stats_cols)[0]

In [ ]:
# raceDB.stats_cols = ['box']+raceDB.stats_cols

In [ ]:
for i,s in enumerate(raceDB.stats_cols):
    print(i,s)

0 inside
1 midfield
2 wide
3 weight
4 DogGrade
5 _dist_last__1
6 _box_last__1
7 _speed_avg_1
8 _split_speed_avg_1
9 _split_margin_avg_1
10 _margin_avg_1
11 _margin_time_avg_1
12 _RunHomeTime_1
13 _run_home_speed_1
14 _first_out_avg_1
15 _pos_out_avg_1
16 _post_change_avg_1
17 _races_1
18 _wins_1
19 _wins_last_1
20 _weight_
21 _min_time_
22 _min_split_time_
23 _last_start_price
24 _last_start_prob
25 dist_dist_last__10
26 dist_box_last__10
27 dist_speed_avg_10
28 dist_split_speed_avg_10
29 dist_split_margin_avg_10
30 dist_margin_avg_10
31 dist_margin_time_avg_10
32 dist_RunHomeTime_10
33 dist_run_home_speed_10
34 dist_first_out_avg_10
35 dist_pos_out_avg_10
36 dist_post_change_avg_10
37 dist_races_10
38 dist_wins_10
39 dist_wins_last_10
40 dist_weight_
41 dist_min_time_
42 dist_min_split_time_
43 dist_last_start_price
44 dist_last_start_prob
45 box_dist_last__10
46 box_box_last__10
47 box_speed_avg_10
48 box_split_speed_avg_10
49 box_split_margin_avg_10
50 box_margin_avg_10
51 box_margi

In [ ]:
from rnn_tools.rnn_classes import Races,Dog, DogInput, Race

In [ ]:
import rnn_tools.raceDB

In [ ]:
stat_list = ['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']


In [ ]:
import glob
enemble_models = []
for model in glob.glob('./esnemble_models/*'):
    print(model)
    files = sorted(glob.glob(f'{model}/*'), key=os.path.getmtime)
    print(files)
    enemble_models.append(files[0])
    
print(enemble_models)
enemble_models  = enemble_models[0:20]

./esnemble_models\ancient-sweep-6
['./esnemble_models\\ancient-sweep-6\\ancient-sweep-6_40.pt', './esnemble_models\\ancient-sweep-6\\ancient-sweep-6_65.pt']
./esnemble_models\comfy-sweep-2
['./esnemble_models\\comfy-sweep-2\\comfy-sweep-2_50.pt']
./esnemble_models\dazzling-sweep-12
['./esnemble_models\\dazzling-sweep-12\\dazzling-sweep-12_50.pt', './esnemble_models\\dazzling-sweep-12\\dazzling-sweep-12_100.pt']
./esnemble_models\divine-sweep-14
['./esnemble_models\\divine-sweep-14\\divine-sweep-14_50.pt', './esnemble_models\\divine-sweep-14\\divine-sweep-14_145.pt']
./esnemble_models\dry-sweep-29
['./esnemble_models\\dry-sweep-29\\dry-sweep-29_80.pt']
./esnemble_models\earnest-sweep-21
['./esnemble_models\\earnest-sweep-21\\earnest-sweep-21_85.pt', './esnemble_models\\earnest-sweep-21\\earnest-sweep-21_240.pt']
./esnemble_models\efficient-sweep-11
['./esnemble_models\\efficient-sweep-11\\efficient-sweep-11_70.pt', './esnemble_models\\efficient-sweep-11\\efficient-sweep-11_245.pt', './e

In [ ]:
importlib.reload(rnn_tools)

<module 'rnn_tools' (<_frozen_importlib_external.NamespaceLoader object at 0x0000017D2C67BBD0>)>

In [ ]:
model_files = enemble_models[0]
model = rnn_classes.GRUNetv4_stacking(raceDB,26,256,num_models=1, num_layers=2,fc0_size=128, fc1_size=64,data_mask_size=26)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
for i,model_file in enumerate(enemble_models):
    print(model_file)
    try:
        model_data = torch.load(model_file, map_location=device)
    except:
        print(model_file, "failed to load")


./esnemble_models\ancient-sweep-6\ancient-sweep-6_40.pt
./esnemble_models\comfy-sweep-2\comfy-sweep-2_50.pt
./esnemble_models\dazzling-sweep-12\dazzling-sweep-12_50.pt
./esnemble_models\divine-sweep-14\divine-sweep-14_50.pt
./esnemble_models\dry-sweep-29\dry-sweep-29_80.pt
./esnemble_models\earnest-sweep-21\earnest-sweep-21_85.pt
./esnemble_models\efficient-sweep-11\efficient-sweep-11_70.pt
./esnemble_models\fancy-sweep-15\fancy-sweep-15_45.pt
./esnemble_models\fast-sweep-10\fast-sweep-10_70.pt
./esnemble_models\fiery-sweep-13\fiery-sweep-13_50.pt
./esnemble_models\glorious-sweep-24\glorious-sweep-24_55.pt
./esnemble_models\graceful-sweep-5\graceful-sweep-5_55.pt
./esnemble_models\kind-sweep-23\kind-sweep-23_50.pt
./esnemble_models\leafy-sweep-3\leafy-sweep-3_25.pt
./esnemble_models\lemon-sweep-18\lemon-sweep-18_80.pt
./esnemble_models\lucky-sweep-7\lucky-sweep-7_45.pt
./esnemble_models\morning-sweep-35\morning-sweep-35_0.pt
./esnemble_models\prime-sweep-19\prime-sweep-19_100.pt
./esne

In [ ]:
model_data = torch.load(model_files, map_location=device)

In [ ]:
model.model_0.load_state_dict(model_data['model_state_dict'])

<All keys matched successfully>

In [ ]:
import gc

In [ ]:
enemble_models

['./esnemble_models\\ancient-sweep-6\\ancient-sweep-6_40.pt',
 './esnemble_models\\comfy-sweep-2\\comfy-sweep-2_50.pt',
 './esnemble_models\\dazzling-sweep-12\\dazzling-sweep-12_50.pt',
 './esnemble_models\\divine-sweep-14\\divine-sweep-14_50.pt',
 './esnemble_models\\dry-sweep-29\\dry-sweep-29_80.pt',
 './esnemble_models\\earnest-sweep-21\\earnest-sweep-21_85.pt',
 './esnemble_models\\efficient-sweep-11\\efficient-sweep-11_70.pt',
 './esnemble_models\\fancy-sweep-15\\fancy-sweep-15_45.pt',
 './esnemble_models\\fast-sweep-10\\fast-sweep-10_70.pt',
 './esnemble_models\\fiery-sweep-13\\fiery-sweep-13_50.pt',
 './esnemble_models\\glorious-sweep-24\\glorious-sweep-24_55.pt',
 './esnemble_models\\graceful-sweep-5\\graceful-sweep-5_55.pt',
 './esnemble_models\\kind-sweep-23\\kind-sweep-23_50.pt',
 './esnemble_models\\leafy-sweep-3\\leafy-sweep-3_25.pt',
 './esnemble_models\\lemon-sweep-18\\lemon-sweep-18_80.pt',
 './esnemble_models\\lucky-sweep-7\\lucky-sweep-7_45.pt',
 './esnemble_models\\m

In [ ]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7 Ensemble", config=config,save_code=False):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      for i in config.stat_list_dict.values():
        print(i)

      reg_stat_mask = []
      for stat,flag in config.stat_list_dict.items():
          stat_flag = flag
          reg_stat_mask = reg_stat_mask+[stat_flag]

      print(reg_stat_mask)
      stat_mask = [1]*6+reg_stat_mask+[0]*80
      data_mask = [0]*26+[1]*20+[1]*20+[1]*20+[1]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask = [1]*6+reg_stat_mask+[0]*20+[0]*20+[0]*20+[0]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask_size = sum(data_mask)
      print(f"{data_mask_size=}")
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
      data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
      # stat_mask = torch.ones_like(stat_mask).type(torch.bool).to(device)

      print(stat_mask)
      print(stat_mask.shape)

      if 'batch_days' in config.keys():
        pass
        raceDB.create_test_split_date(config['training_date_end'],val_date='2023-07-01')
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask,data_mask=data_mask)

      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          input_size = raceDB.batches['packed_x'][0].data[0].shape[0]

      # for race in raceDB.racesDict.values():
      #     for dog_input in race.dogs:
      #       dog_input.stats_masked = dog_input.stats.masked_select(stat_mask) 
      ensemble_models = config['ensemble_models']
      ensemble_models = ensemble_models
      num_batches = len(ensemble_models)
      print(f"{input_size=}")
      print(f"Num batches {len(raceDB.batches['packed_x'])}")
      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv4_stacking(raceDB,input_size,config['hidden_size'],num_models=num_batches, num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      with torch.no_grad():
        for i,model_dict in tqdm(enumerate(ensemble_models)):
          model_loc = model_dict
      #     # print(model_dict)
          
          raceDB.dogsDict['nullDog'].input.hidden_out = (-torch.ones(256+64)).to('cuda:0')
          model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
          # print(model_data['model_state_dict'].keys())
          
          model.model_list[i].load_state_dict(model_data['model_state_dict'], strict=True)
          training_testing_gru_ensemble_pretrained.load_model_cache(raceDB,model.model_list[i])
          model.model_list[i].hidden_dict = {}
          torch.cuda.empty_cache()
          
      #   # asad
          
      raceDB.del_hidden()
      gc.collect
      torch.cuda.empty_cache()
      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')
      optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      model = model.to(device)
      # print(model)y6

      # stop
      # and use them to train the model
      # wandb.watch(model, log='all')
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru_ensemble_pretrained.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      # dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        # raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
pre_trained_bases = {}

pre_trained_bases[0] = {'model_name':'stellar-pine-41', 'version':120}
pre_trained_bases[1] = {'model_name':'dulcet-feather-42', 'version':60}
pre_trained_bases[2] = {'model_name':'autumn-blaze-43', 'version':100}
pre_trained_bases[3] = {'model_name':'exalted-paper-45', 'version':40}
pre_trained_bases[4] = {'model_name':'sweet-resonance-46', 'version':20}

stat_list_dict = {'_dist_last__1':1,
    '_box_last__1':1,
    '_speed_avg_1':1,
    '_split_speed_avg_1':1,
    '_split_margin_avg_1':1,
    '_margin_avg_1':1,
    '_margin_time_avg_1':1,
    '_RunHomeTime_1':1,
    '_run_home_speed_1':1,
    '_first_out_avg_1':1,
    '_pos_out_avg_1':1,
    '_post_change_avg_1':1,
    '_races_1':1,
    '_wins_1':1,
    '_wins_last_1':1,
    '_weight_':1,
    '_min_time_':1,
    '_min_split_time_':1,
    '_last_start_price':1,
    '_last_start_prob':1,
}

wandb_config_static = {
    'hidden_size': 256,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 1500,
    'learning_rate': 0.001,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2023-01-01',
    'notes':'GRU, with basic add on data looped in',
    'batch_days': 90,
    'stat_list_dict': stat_list_dict,
    'ensemble_models':enemble_models,
}

In [ ]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
raceDB.race_prices_to_prob()

c:\Users\Nick\Documents\GitHub\grvmodel\python\rnn_tools\rnn_classes.py:389: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r.prob = torch.tensor((1/(torch.tensor(r.start_prices)+0.0001))/((1/torch.tensor(r.start_prices)).sum()+0.0001),device=self.device)


In [ ]:
raceDB.latest_date

Timestamp('2024-01-31 00:00:00')

In [1]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_ensemble_pretrained)
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# all_price_df = model_pipeline(raceDB,config=wandb_config_static,sweep=False)


NameError: name 'importlib' is not defined

In [ ]:
raceDB.create_test_split_date('2023-01-01',val_date='2023-07-01')

Train examples 15593, Test examples 2423, Val examples 2833


In [ ]:
len_test = len(raceDB.test_dog_ids)
test_idx = range(0,len_test)
race = raceDB.get_test_input(1)


TypeError: operator.itemgetter() argument after * must be an iterable, not int

In [ ]:
races = [self.racesDict[x] for x in self.test_race_ids]

1307

In [ ]:
wandb_config_static['batch_days'] = 600
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static['batch_days'] = 180
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static['batch_days'] = 90
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static['batch_days'] = 60
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static['batch_days'] = 630
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)


In [ ]:
model

In [ ]:
importlib.reload(training_testing_gru_extra_data)
training_testing_gru_extra_data.validate_model_v3(model,raceDB,config=wandb_config_static)

In [ ]:
config = wandb_config_static

In [ ]:
model = rnn_classes.LSTMNetv3_BN_double(22,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])

In [ ]:
model = model.to('cuda:0')

In [ ]:
sweep_config = {"method": "grid"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "batch_days": {"values": [90,180,365,550,10000]},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "input_type": {"values": ['basic']},
    "num_layers": {"values": [2]},
    'hidden_size':{'values':[128]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "stats":{"value": raceDB.stats_cols},
    "races":{"value": states},
    "batch_size":{"value": 10},
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [1000]},
        "validation_split": {"value": 0.1},
        "training_date_end": {"values": 
                                [
                                '2023-04-30',
                                '2023-02-28',
                                '2022-11-30',
                                '2021-11-30'
                                ]},
        "learning_rate": {"values": [0.0003, 0.0001]},
        "label_smoothing": {"values": [0.01,0.0]},
        "loss": {
            "values": [ "CEL"],
        },
    'dist_last__1': {'values': [0, 1]},
    'box_last__1': {'values': [0, 1]},
    'speed_avg_1': {'values': [0, 1]},
    'split_speed_v1_1': {'values': [0, 1]},
    'split_speed_avg_1': {'values': [0, 1]},
    'split_margin_avg_1': {'values': [0, 1]},
    'margin_avg_1': {'values': [0, 1]},
    'margin_time_avg_1': {'values': [0, 1]},
    'RunHomeTime_1': {'values': [0, 1]},
    'run_home_speed_1': {'values': [0, 1]},
    'run_home_speed_v1_1': {'values': [0, 1]},
    'first_out_avg_1': {'values': [0, 1]},
    'pos_out_avg_1': {'values': [0, 1]},
    'post_change_avg_1': {'values': [0, 1]},
    'races_1': {'values': [0, 1]},
    'wins_1': {'values': [0, 1]},
    'wins_last_1': {'values': [0, 1]},
    'weight_': {'values': [0, 1]},
    'min_time_': {'values': [0, 1]},
    'min_split_time_': {'values': [0, 1]},
    'min_split_time_v1': {'values': [0, 1]},
    'last_start_price': {'values': [0, 1]},
    'last_start_prob': {'values': [0, 1]}
}
)

import pprint

pprint.pprint(sweep_config)


sweep_config

In [ ]:
# wandb.agent('x9mz21wl', function=model_pipeline, count=100, project='NEW GRU-sweeps')

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Variable Sweeps")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=1)